# Modelos Lineales - Parte 1. Regresión

In [ ]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import sys

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "05_ModelosLineales"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "plots", CHAPTER_ID)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    os.makedirs(IMAGES_PATH, exist_ok=True)
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
# import warnings
# warnings.filterwarnings(action="ignore", message="^internal gelsd")

## Regresión lineal (univariada).

Vuelve a llamar Alex. Está agradecido. Pudo impresionar a su jefe gracias a lo que le mandamos de Random Forests, Neural Networks, etc. Lo nombraron *Chief Artificial Intelligence Officer*, a cargo de departamento de AI de la inmobiliaria. Pero está de vuelta en problemas (si no, creo que no llamaría). 

Resulta que hace un par de fin de semanas, el jefe de Alex estaba jugando al golf con otros dueños de inmobiliarias, y se mandó la parte con su recientemente creado departamento de Inteligencia Artificial, y les contó todo lo que habían logrado (gracias a nosotrxs) con respecto a la predicción de precios de casas en distritos de California. Pero se vio en un brete cuando uno de sus colegas le pidió alguna explicación respecto al funcionamiento de la predicción. ¿Qué variables son las más importantes? ¿Cómo hace la predicción?

El lunes siguiente, el jefe volvió a la inmobiliaria y entró precipitadamente en la oficina de Alex para pedirle que arme un equipo que pudiera darle algún sentido a las predicciones que habían logrado antes. Le dio permiso para contratar a una persona. Alex está contento porque logró meter a su novia, Brenda, como *Senior Parameter Explorer*, pero tienen que producir algún modelo que sea más fácil de entender para el jefe, sobre todo para que no se deschave que Brenda en realidad estudio diseño gráfico.

In [ ]:
# Volvamos a leer los datos de California
HOUSING_PATH = os.path.join(".", "datasets", "housing")

if 'google.colab' in sys.modules:
    
    import tarfile

    DOWNLOAD_ROOT = "https://raw.githubusercontent.com/IAI-UNSAM/ML_UNSAM/master/"
    HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

    !mkdir -p ./datasets/housing

    def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
        os.makedirs(housing_path, exist_ok=True)
        tgz_path = os.path.join(housing_path, "housing.tgz")
        #urllib.request.urlretrieve(housing_url, tgz_path)
        !wget https://raw.githubusercontent.com/IAI-UNSAM/ML_UNSAM/master/datasets/housing/housing.tgz -P {housing_path}
        housing_tgz = tarfile.open(tgz_path)
        housing_tgz.extractall(path=housing_path)
        housing_tgz.close()
    
    # Corramos la función
    fetch_housing_data()
    
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()

In [ ]:
housing.info()

In [ ]:
housing.describe().round(2)

### Coeficiente de Pearson

In [ ]:
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
# Vamos a quedarnos solo con la mediana del ingreso y el valor de las casas.
x = housing.median_income
y = housing.median_house_value

Ahora nos podemos preguntar de dónde sale ese valor de 0.688. Ya lo mencionamos, pero ahora que sabemos algo más de distribuciones, podemos volver a verlo.

Si usamos la siguiente notación:

$$
\begin{array}{lll}
\mu_X &=& \mathbb{E}(X)\\
\sigma^2_X &=& \mathbb{E}\left[(X - \mathbb{E}(X))^2\right] = \mathrm{var}(X)\;\;,
\end{array}
$$
el coeficiente de correlación se define como 

$$
\rho_{XY} = \mathbb{E}\left[\left(\frac{X - \mu_X}{\sigma_X}\right)\left(\frac{Y - \mu_Y}{\sigma_Y}\right)\right]\;\;.
$$

El valor de expectación $\mathbb{E}$ lo definimos para las distribuciones hace unas clases. Para el caso continuo:

$$
\mathbb{E}(X) = \int x f_X(x) \mathrm{d}x\;\;.
$$

Ahora bien, ¿qué pasa si no conocemos, como ahora, $f_X$? ¿Qué pasa si solo tenemos una muestra de esa distribución?

Entonces, tenemos que usar los datos para *estimar* el valor esperado, $\mathbb{E}(X)$. Para eso, formamos un *estadístico* (*statistic*, en singular, en inglés). Es decir, una función de los datos.

$$
\bar{X} = \frac{1}{N}\sum_{i=1}^N x_i\;\;.
$$

Cuando un estadístico se usa para aproximar el valor de una característica poblacional no conocido, se lo llama un *estimador*. En este caso el estadístico $\bar{X}$ es un *estimador* del valor de expectación de la distribución, y a veces vamos a usar la notación:

$$
\hat{\mu_X} = \bar{X}\;\;.
$$

También tenemos *estimadores* de la varianza y de la covarianza:

$$
\hat{\sigma}_X^2 = \frac{1}{N - 1}\sum_{i=1}^N (x_i - \bar{X})^2\;\;,
$$
y
$$
\hat{\mathrm{cov}}_{XY} = \frac{1}{N - 1}\sum_{i=1}^N (x_i - \bar{X})(y_i - \bar{Y})\;\;.
$$

Con todo esto, podemos calcular un estimador del coeficiente de correlación:

$$
\hat{\rho_{XY}} = r = \frac{\hat{\mathrm{cov}}_{XY}}{\hat{\sigma}_X \hat{\sigma}_Y}\;\;,
$$
que se conoce como el *coeficiente de correlación de Pearson*.

In [ ]:
# Calculamos los estimadores de los valores medios
Xbar = np.sum(x)/len(x)
Ybar = np.sum(y)/len(y)

# Calculamos los estimadores de los desvíos estándar (es decir, sqrt(cov))
sigmaX = np.sqrt(np.sum((x - Xbar)**2) / (len(x) - 1))
sigmaY = np.sqrt(np.sum((y - Ybar)**2) / (len(y) - 1))

covXY = np.sum((x - Xbar) * (y - Ybar)) / (len(x) - 1)

# Coeficiente de Pearson
r = covXY / (sigmaX * sigmaY)

print('El coeficiente de Pearson es: {:.3f}'.format(r))

En `numpy` tenemos funciones y métodos que hacen esto: `np.mean`, `np.std`, `np.cov`, `np.corrcoef`.

De ahora en más, usen esas implementaciones, porque en el fondo, `numpy` corre código en C y es mucho más rápido.

In [ ]:
corr_matrix['median_house_value'].sort_values(ascending=False)

El coeficiente de correlación toma valores $-1 < \rho < 1$, donde -1 indica una anticorrelación perfecta, y 1 indica una correlación perfecta.

El estimador, $r$, es muy sensible a puntos aberrantes (*outliers*), por lo que hay que mirar un poco los datos antes de calcular los coeficientes y ya.

In [ ]:
plt.plot(x, y, '.', alpha=0.1)
plt.xlabel('Mediana del ingreso por distrito')
plt.ylabel('Mediana del precio de la casa')

In [ ]:
# Veamos como cambia $r$ si sacamos los distritos con precios que saturan
i = y < np.max(y)

plt.plot(x[i], y[i], '.', alpha=0.1)
plt.xlabel('Mediana del ingreso por distrito')
plt.ylabel('Mediana del precio de la casa')


r = np.corrcoef(x[i], y[i])[0, 1]
print('El coeficiente de Pearson, con datos corregidos, es: {:.3f}'.format(r))

Claro. Los puntos saturados, arriba a la derecha ayudan mucho a aumentar $r$ de forma espúrea.

### Modelo lineal sencillo.

El ingreso no es una variable ideal para determinar el precio de una casa, pero es lo mejor que tenemos, de manera individual. ¿Qué modelo podemos proponerles?

Brenda recordó algo de algunas clases de matemática que tuvo en la carrera y pensó que podía armar un modelo que tuviera esta pinta:

$$
y = a * x + b\;\;,
$$
es decir, una relación lineal entre ambas variables. En este tipo de modelo, la variable $x$ se conoce con el nombre de variable predictora. Este modelo tiene dos *parámetros*: $a$ y $b$, la pendiente y ordenada al origen, respectivamente.

Brenda encuentra en sus apuntes viejos una expresión para *ajustar* los parámetros:

$$
\begin{array}{lll}
\hat{a} &=& \sum_{i=0}^N (x_i - \bar{X}) (y_i - \bar{Y}) \left[\sum_{i=0}^N (x_i - \bar{X})^2\right]^{-1}\\
\hat{b} &=& \bar{Y} - \hat{a}\bar{X}
\end{array}
$$

Comparando con las ecuaciones de arriba, vemos que

$$
\hat{a} = \frac{\hat{\mathrm{cov}}_{XY}}{\hat{\mathrm{var}}(X)}
$$

In [ ]:
# Calculemos
ahat = np.cov(x[i], y[i])[0,1] / np.var(x[i])
bhat = np.mean(y[i]) - ahat * np.mean(x[i])

print('Los ajustes dan')
print('a = {:.3f}'.format(ahat))
print('b = {:.3f}'.format(bhat))

In [ ]:
plt.plot(x[i], y[i], '.', alpha=0.1)

xx = np.linspace(x[i].min(), x[i].max(), 3)
plt.plot(xx, xx * ahat + bhat, 'r-')
plt.xlabel('Mediana del ingreso por distrito')
plt.ylabel('Mediana del precio de la casa [$]')

In [ ]:
# Podemos ver los residuos, y calcular su dispersión
res = y[i] - (ahat * x[i] - bhat)
plt.plot(x[i], res, '.', alpha=0.1)
plt.axhline(0, color='0.5', ls=':')

print('Los residuos tienen una dispersión de $ {:.3f}'.format(res.std()))

**Pregunta**

- ¿Estamos contentos con el ajuste? ¿Por qué?

### ¿De dónde viene la fórmula de regresión lineal? (o ¿qué condiciones se tienen que cumplir para que la cosa funcione?)

Formalizando un poco más la cosa, podemos recordar que habíamos dicho que un modelo podía escribirse:

$$
y_i = m_i + \epsilon_i\;\;,
$$
donde $\epsilon_i$ es el término de error.

De ahora en más, vamos a llamar:
* $t_i$ a los valores de $Y$ que queremos reproducir,
* $y(x, \omega)$ al modelo, que dependerá de una vector de parámetros $\boldsymbol{\omega}$.

En este caso, estamos tomando, $y(x, \boldsymbol{\omega}) = \omega_0 + \omega_1 * x$.

En general, es más conveniente usar el logaritmo de la verosimilitud. Como el logaritmo es una función creciente, el máximo de la verosimilitud coincide con el máximo de su logaritmo.

En este caso, obtenemos:

$$
\ln p(\boldsymbol{t} | \mathbf{x}, \boldsymbol{\omega}, \beta) = -\frac{\beta}{2} \sum_{i=1}^{N} \left\{\left(y(x_i, \boldsymbol{\omega}) - t_i\right)^2\right\} + \frac{N}{2}\ln\beta - \frac{N}{2}\ln 2\pi\;\;.
$$

Como los últimos dos términos son constantes con respecto a los parámetros, podemos obviarlos a la hora de maximizar la verosimilitud. La tarea es, entonces, equivalente a minimizar:

$$
E(\boldsymbol{\omega}) = \frac{1}{2} \sum_{i=1}^{N} \left\{y(x_i, \boldsymbol{\omega}) - t_i\right\}^2\;\;,
$$
que se conoce como *error cuadrático medio*.

Muchas veces, es más simpático tener una función de error que tenga las mismas unidades que los datos. Por eso, definimos la *raíz del error cuadrático medio* (*root-mean-square error*, o *RMS*):

$$
E_{RMS} = \sqrt{2 * E(\boldsymbol{\omega})/N}
$$

***

**Ejercicio**

* Mostrar usando la expresión del error cuadrático medio, y el modelo lineal, que los valores de los parámetros $a$ y $b$ que maximizan la verosimilitud son los que aparecen arriba.

***

### Verificación de las hipótesis

Volvamos al caso de California y estudiemos con más detalle lo que pasa con los residuos.

En primer lugar, podemos calcular el error cuadrático medio (y su raíz) para el ajuste que hicimos arriba?

In [ ]:
E = 0.5 * np.sum(res**2)

print('El error cuadrático medio es {:.3f}'.format(E))
print('El root-mean-square error es {:.3f}'.format(np.sqrt(2 * E / len(x[i]))))

***
**Pregunta**

* ¿Cómo se les ocurre que podemos estudiar el cumplimiento de las hipótesis?
***


In [ ]:
import scipy.stats as st

In [ ]:
# Veamos la distribución de los residuos
a = plt.hist(res, 100, density=True)

# Podemos graficar una Gaussiana encima
# Vamos a plotear la _mejor_
xx = np.linspace(res.min(), res.max(), 100)
plt.plot(xx, st.norm.pdf(xx, res.mean(), res.std()), '-')

In [ ]:
import scipy.stats as st
st.probplot((res - res.mean())/res.std(), plot=plt.gca())

In [ ]:
st.probplot(st.norm.rvs(0, 1, size=1000), plot=plt.gca())

### Regresión lineal con datos sintéticos (o el fin de la era de inferencia).

## Regresión lineal multivariada

### Regresión polinomial

### Sobreajuste

### Regularización